In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

##Import any other packages you may need here
import tensorflow as tf
from skimage.transform import resize
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
#     print('Load file {} ...'.format(filename))
#     ds = pydicom.dcmread(filename)       
#     img = ds.pixel_array

    # Load file and reads image
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    # Confirm appropriate body part, modality, and patient position
    if (ds.BodyPartExamined != 'CHEST') or (ds.Modality != 'DX') or (ds.PatientPosition not in ['AP', 'PA']):
        print('Warning! There is invalid data in file: '.format(filename))
        print('')
        return None, 0, 0
    
    # Calculate mean and standard deviation of image
    img_mean = np.mean(img)
    img_std = np.std(img)
    
    # Print values of key attributes
    print('Modality is ', ds.Modality)
    print('Study Description is ', ds.StudyDescription)
    print('Patient Id is ', ds.PatientID)
    print('Patient Sex is ', ds.PatientSex)
    print('Patient Age is ', ds.PatientAge)
    print('Patient Position:', ds.PatientPosition)
    print('Body Part Examined:', ds.BodyPartExamined)
    
    return img, img_mean, img_std
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    # todo
    proc_img = img.copy()
    proc_img = resize(img, img_size)
    proc_img = (proc_img - img_mean)/img_std
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    # Read json file with trained model
    json_file = open(model_path, "r")
    loaded_model = json_file.read()
    json_file.close()
    
    # Load trained model
    model = model_from_json(loaded_model)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    pred_Y = model.predict(img, batch_size = 1, verbose = True)
    
    if pred_Y[0][0] > thresh:
        prediction = 'Pneumonia present'
    else:
        prediction = 'No Pneumonia present'

    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" #path to saved model
weight_path = "{}_my_model.best.hdf5".format('xray_class') #path to saved best weights

IMG_SIZE = (1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path)  #loads model
thresh = 0.42 #loads the threshold they chose for model classification

# use the .dcm files to test your prediction
# img_mean is the mean image value they used during training preprocessing
# img_std is the std dev image value they used during training preprocessing

for i in test_dicoms:
    
    img = np.array([])
    img, img_mean, img_std = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model, img_proc, thresh)
    print(pred)
    print('')

Load file test1.dcm ...
Modality is  DX
Study Description is  No Finding
Patient Id is  2
Patient Sex is  M
Patient Age is  81
Patient Position: PA
Body Part Examined: CHEST
1/1 [==============================] - 2s 2s/step
No Pneumonia present

Load file test2.dcm ...
Modality is  DX
Study Description is  Cardiomegaly
Patient Id is  1
Patient Sex is  M
Patient Age is  58
Patient Position: AP
Body Part Examined: CHEST
1/1 [==============================] - 0s 25ms/step
No Pneumonia present

Load file test3.dcm ...
Modality is  DX
Study Description is  Effusion
Patient Id is  61
Patient Sex is  M
Patient Age is  77
Patient Position: AP
Body Part Examined: CHEST
1/1 [==============================] - 0s 25ms/step
No Pneumonia present

Load file test4.dcm ...
Warning! There is invalid data in file: 

Load file test5.dcm ...
Warning! There is invalid data in file: 

Load file test6.dcm ...
Warning! There is invalid data in file: 

